# Hack Day Analysis

- Try loading our models from last night
- Try out tiny stories models. 

## Get our model from last night

In [2]:
import wandb
run = wandb.init()
artifact = run.use_artifact('jbloom/mats_sae_training_language_models/sae:v1', type='model')
artifact_dir = artifact.download()
wandb.finish()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jbloom. Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact sae:v1, 128.13MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:0.4


In [9]:
weights = torch.load(artifact_dir + '/sae.pt')
weights["W_enc"].shape

torch.Size([512, 32768])

In [40]:
import torch 
from dataclasses import dataclass
from sae_training.SAE import SAE

@dataclass
class SAEConfig():
    d_in = weights["W_enc"].shape[0]
    d_sae = weights["W_enc"].shape[1]
    device = "cuda"
    dtype = torch.float32
    l1_coefficient = 0.0001

sae_cfg = SAEConfig()
SAE = SAE(sae_cfg)
SAE.load_state_dict(weights)

<All keys matched successfully>

In [24]:
from transformer_lens import HookedTransformer

model = HookedTransformer.from_pretrained("gelu-2l")

Loaded pretrained model gelu-2l into HookedTransformer


In [34]:
# load some data
from torch.utils.data import DataLoader
from sae_training.activations_buffer import DataLoaderBuffer

@dataclass
class DataLoaderBufferConfig():
    d_in: int
    d_sae: int
    device: str
    dtype: torch.dtype
    batch_size: int
    n_workers: int
    context_size: int = 128
    store_batch_size: int = 16
    n_batches_in_buffer: int = 10
    hook_point: str = "blocks.0.hook_mlp_out"
    


cfg = DataLoaderBufferConfig(
    d_in=weights["W_enc"].shape[0],
    d_sae=weights["W_enc"].shape[1],
    device="cuda",
    dtype=torch.float32,
    batch_size=128,
    n_workers=0,
)

activations_buffer = DataLoaderBuffer(
    cfg, model, data_path="NeelNanda/c4-tokenized-2b",
)

def get_new_dataloader(data_loader_buffer, batch_size):
    buffer = data_loader_buffer.get_buffer()
    dataloader = iter(DataLoader(buffer, batch_size=batch_size, shuffle=True))
    n_remaining_batches_in_buffer = len(dataloader)
    return dataloader, n_remaining_batches_in_buffer

dataloader, n_remaining_batches_in_buffer = get_new_dataloader(activations_buffer, 128)


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

In [36]:
tokens = next(dataloader)

In [57]:
activations = activations_buffer.get_buffer()
sae_out, feature_acts, loss, mse_loss, l1_loss = SAE(activations)
(feature_acts > 0).float().sum(0).mean()

tensor(61.6924, device='cuda:0')

In [42]:
mse_loss

tensor(11.5401, device='cuda:0', grad_fn=<MeanBackward1>)

In [51]:
(feature_acts > 0).float().sum(1).mean()

tensor(100.7457, device='cuda:0')

tensor(62.9661, device='cuda:0')